## Tensorflow2: Linear Regression 

### This notebook is a TensorFlow port of the PyTorch notebook: https://jovian.ml/aakashns/02-linear-regression

Despite the structural differences in Tensorflow and PyTorch, I have tried to port the torch notebooks to tensorflow, which helps in learning both frameworks along with the course [PyTorch: Zero to GANs](https://jovian.ml/aakashns/01-pytorch-basics) by Aakash

#### Part 2 of "Tensorflow: Zero to GANs"

1. [Tensorflow Basics: Tensors & Gradients](https://jovian.ml/kartik.godawat/01-tensorflow-basics)
2. [Linear Regression & Gradient Descent](https://jovian.ml/kartik.godawat/02-tf-linear-regression)
3. [Image Classfication using Logistic Regression](https://jovian.ml/kartik.godawat/03-tf-logistic-regression) 
4. [Training Deep Neural Networks on a GPU](https://jovian.ml/kartik.godawat/04-tf-feedforward-nn)


Continuing where the [previous tutorial](https:|//jovian.ml/kartik.godawat/01-tensorflow-basics) left off, we'll discuss one of the foundational  algorithms of machine learning in this post: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

## System Setup

We begin by importing Numpy and PyTorch:

## Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to TensorFlow tensors as follows:

In [4]:
inputs = tf.convert_to_tensor(inputs)
targets = tf.convert_to_tensor(targets)
print(inputs)
print(targets)

tf.Tensor(
[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]], shape=(5, 2), dtype=float32)


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [5]:
w = tf.Variable(tf.random.normal([2, 3]), trainable=True)
b = tf.Variable(tf.random.normal([2]))
print(w)
print(b)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.4917677 , -1.2063944 ,  0.3641779 ],
       [-0.8602699 , -0.9507346 , -0.33335298]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([-0.82898927, -0.83526826], dtype=float32)>


`tf.random.normal` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ tf.transpose(w) + b

`@` represents matrix multiplication, and the `tf.transpose(x)` method returns the transpose of a tensor `x`.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tf.Tensor(
[[-101.89681 -141.66837]
 [-128.43518 -184.11906]
 [-184.14732 -222.41165]
 [ -89.38967 -141.79845]
 [-125.08237 -174.79913]], shape=(5, 2), dtype=float32)


Let's compare the predictions of our model with the actual targets.

In [8]:
# Compare with targets
print(targets)

tf.Tensor(
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]], shape=(5, 2), dtype=float32)


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return tf.math.reduce_sum(diff * diff) / tf.size(diff, out_type=tf.float32)

`tf.math.reduce_sum` returns the sum of all the elements in a tensor, and the `tf.size` method returns the number of elements in a tensor. Note that we need to pass `out_type=tf.float32` to ensure denominator matches the dtype of numerator Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tf.Tensor(59582.27, shape=(), dtype=float32)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute gradients

Computation of gradients in Tensorflow2.x is done by using `tf.GradientTape` to *record* all the operations on a tape, and then calculate the derivative of y wrt x using `tape.gradient(y,x)`. In order for tensorflow to calculate the derivates, loss calculation should be within GradientTape block. Also, since we declared our `w` and `b` as `tf.Variable`, they're automatically watched by gradient tape. In order to track a tensor, we need to watch it by using `tape.watch(var)` before calculating the loss

In [11]:
with tf.GradientTape(persistent=True) as tp:
    preds = model(inputs)
    loss = mse(preds, targets)

In [12]:
print(w)
print(tp.gradient(loss, w))

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.4917677 , -1.2063944 ,  0.3641779 ],
       [-0.8602699 , -0.9507346 , -0.33335298]], dtype=float32)>
tf.Tensor(
[[-16811.664 -19263.357 -11572.629]
 [-22165.605 -24558.094 -15028.943]], shape=(2, 3), dtype=float32)


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

Let's implement the above step by step.

In [13]:
# Generate predictions and Calculate loss
with tf.GradientTape(persistent=True) as tape:
    preds = model(inputs)
    loss = mse(preds, targets)
# Compute gradient w.r.t the weights and biases
w_grad = tape.gradient(loss,w)
b_grad = tape.gradient(loss,b)
# Adjust the weights
w.assign(w - w_grad * 1e-5)
b.assign(b - b_grad * 1e-5)
print(loss)

tf.Tensor(59582.27, shape=(), dtype=float32)


A few things to note above:

- This implementation is slightly different than PyTorch's no_grad implementation

- We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm.

- Gradient reset to zero isn't needed here in contrast to PyTorch implementation

Let's take a look at the new weights and biases.

In [14]:
print(w)
print(b)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.32365108, -1.0137608 ,  0.4799042 ],
       [-0.6386138 , -0.7051537 , -0.18306355]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([-0.8269694 , -0.83261865], dtype=float32)>


With the new weights and biases, the model should have lower loss.


In [15]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tf.Tensor(40388.848, shape=(), dtype=float32)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [16]:
# Train for 100 epochs
tape = None
for i in range(100):
    with tf.GradientTape(persistent=True) as tape:
        preds = model(inputs)
        loss = mse(preds, targets)        
    w.assign(w - tape.gradient(loss,w) * 1e-5)
    b.assign(b - tape.gradient(loss,b) * 1e-5)
    
#     print(w,b)

Once again, let's verify that the loss is now lower:

In [17]:
preds = model(inputs)
loss = mse(preds, targets)  
print(loss)

tf.Tensor(219.09676, shape=(), dtype=float32)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [18]:
preds

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[ 62.05404 ,  73.82271 ],
       [ 89.286644, 100.95453 ],
       [ 94.72926 , 126.660446],
       [ 50.675346,  58.026142],
       [ 96.80508 , 107.20921 ]], dtype=float32)>

In [19]:
targets

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)>

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs. 


## Linear regression using Tensorflow + Keras built-ins 

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , Tensorflow2 has several built-in functions and classes to make it easy to create and train models.

*Keras* is a higher level libary available within tensorflow and can be imported using using `tf.keras`

As before, we represent the inputs and targets and matrices.

In [20]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')
inputs = tf.convert_to_tensor(inputs)
targets = tf.convert_to_tensor(targets)

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

## Dataset

We'll use `tf.data` api , which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in tensorflow.

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((inputs,targets))

In [22]:
dataset

<TensorSliceDataset shapes: ((3,), (2,)), types: (tf.float32, tf.float32)>

To split the data into batches of a predefined size while training, we use `dataset.batch()`

In [23]:
BATCH_SIZE = 5
dataset = dataset.batch(BATCH_SIZE)

In [24]:
dataset

<BatchDataset shapes: ((None, 3), (None, 2)), types: (tf.float32, tf.float32)>

You can see above that dataset has now been converted to `BatchDataset`. The data now can be accessed using simple for loop like this:

In [25]:
for x, y in dataset:
    print(x)
    print(y)

tf.Tensor(
[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]], shape=(5, 2), dtype=float32)
tf.Tensor(
[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]], shape=(5, 2), dtype=float32)
tf.Tensor(
[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]], shape=(5, 2), dtype=float32)


In each iteration, the data loader returns one batch of data, with the given batch size.`dataset.shuffl()`, can be called to shuffle the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

# Create model
Instead of initializing weights and biases manually, we can use `tf.keras` apis to create a model

In [26]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(2, input_shape=(3,))
])
    

To view the weights and biases in a model, we can use `model.weights`

In [27]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.1928345 , -0.40553868],
        [ 0.55817056, -0.108769  ],
        [ 0.32542884, -0.8425481 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

We can use the model to generate predictions in the exact same way as before:

In [28]:
preds = model(inputs)
preds

<tf.Tensor: shape=(15, 2), dtype=float32, numpy=
array([[  37.31395 ,  -73.121414],
       [  52.398518, -100.39877 ],
       [  76.89312 ,  -98.7247  ],
       [  16.373081,  -77.21629 ],
       [  63.058815,  -97.40236 ],
       [  37.31395 ,  -73.121414],
       [  52.398518, -100.39877 ],
       [  76.89312 ,  -98.7247  ],
       [  16.373081,  -77.21629 ],
       [  63.058815,  -97.40236 ],
       [  37.31395 ,  -73.121414],
       [  52.398518, -100.39877 ],
       [  76.89312 ,  -98.7247  ],
       [  16.373081,  -77.21629 ],
       [  63.058815,  -97.40236 ]], dtype=float32)>

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [29]:
loss_fn = tf.keras.losses.MeanSquaredError()

Let's compute the loss for the current predictions of our model.

In [30]:
loss = loss_fn(model(inputs),targets)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=17918.404>

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `tf.keras.optimizers.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [31]:
opt = tf.keras.optimizers.SGD(learning_rate=1e-5)

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [32]:
def fit(num_epochs, model, loss_fn, opt, train_dataset):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for x, y in train_dataset:
            # Compute loss
            with tf.GradientTape() as tt:
                preds = model(x)
                loss = loss_fn(preds, y)
            var_list_fn =  model.trainable_weights
            # Compute gradients
            grads = tt.gradient(loss, var_list_fn)
            # Apply gradients
            opt.apply_gradients(zip(grads, var_list_fn))
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss))

Some things to note above:

- We use the`tf.data.Dataset` defined earlier to get batches of data for every iteration.

- Instead of updating parameters (weights and biases) manually, we use `opt.apply_gradients` after calculating all the gradients. The list of all variables to update can be fetched from `model.trainable_weights`

We've also added a log statement which prints the loss from the last batch of data for every 10th epoch.

Let's generate predictions using our model and verify that they're close to our targets.

In [33]:
fit(100, model, loss_fn, opt, dataset)

Epoch [10/100], Loss: 195.0150
Epoch [20/100], Loss: 152.1016
Epoch [30/100], Loss: 121.8918
Epoch [40/100], Loss: 100.2858
Epoch [50/100], Loss: 84.6093
Epoch [60/100], Loss: 73.0330
Epoch [70/100], Loss: 64.3043
Epoch [80/100], Loss: 57.5656
Epoch [90/100], Loss: 52.2282
Epoch [100/100], Loss: 47.8881


In [34]:
model(x)

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[ 57.768314,  71.83418 ],
       [ 81.152054,  94.82893 ],
       [120.12243 , 143.79146 ],
       [ 24.440329,  45.17917 ],
       [ 98.12515 , 104.213486]], dtype=float32)>

In [35]:
y

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)>

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.

## Commit and update the notebook
As a final step, we can record a new version of the notebook using the `jovian` library.

In [1]:
import jovian

In [ ]:
jovian.commit(filename="02-tf-linear-regression", environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
